In [1]:
#pre-process train_data and label
import json
f = open('/Users/enn/Desktop/nlp/project1/project-files/train_total.json')
train_data = json.load(f)

train = []
y_train = []

for key, value in train_data.items():
    train.append(value['text'])
    y_train.append(value['label'])
for item in train:
    item = item.replace('\n', '')

print(len(y_train))

2766


In [2]:
#pre-process dev_data and label
import json
f = open('/Users/enn/Desktop/nlp/project1/project-files/dev.json')
dev_data = json.load(f)

dev = []
y_dev = []

for key, value in dev_data.items():
    dev.append(value['text'])
    y_dev.append(value['label'])
for item in dev:
    item = item.replace('\n', '')

print(len(y_dev))

100


In [3]:
#pre-process test data
f = open('/Users/enn/Desktop/nlp/project1/project-files/test-unlabelled.json')
test_data = json.load(f)

test = []
test_title = []
for key, value in test_data.items():
    test.append(value['text'])
    test_title.append(key)
for item in test:
    item = item.replace('\n', '')

print(len(test_title))

1410


In [4]:
#delete symbols in train & dev data
import re

string = ''
for event in train:
    string += event
for item in dev:
    string += item
string = re.sub(r'[^\w\s]','', string)
#print(string)

In [5]:
from collections import defaultdict, Counter

#wordtoken & abandon words whose frequency is 1. Count the size of vocabulary
c = Counter(string.split())
C = sorted(c.most_common(), key=lambda x:x[1])
#print(c)
print(len(C))
stop = []
for item in C:
    if item[1] == 1:
        stop.append(item[0])
print(len(stop))

46598
21190


In [6]:
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict, Counter

tt = TweetTokenizer()
stopwords = set(stopwords.words('english') + stop)
#print(len(stopwords))
def preprocess_events(events):
    tokenized_events = []
    for event in events:
        event = re.sub(r'[^\w\s]','', event)
        all_tokens = [token for token in tt.tokenize(event) if not token in stopwords]
        tokenized_events.append(all_tokens)
    return tokenized_events
#print(len(train))
x_train = preprocess_events(train)
x_dev = preprocess_events(dev)
x_test = preprocess_events(test)
#print(len(x_train))
#print(train)

In [7]:
import json
from numpy import dot
from numpy.linalg import norm
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from sklearn.utils import shuffle
import os
import numpy as np

tokenizer = Tokenizer(oov_token = "<UNK>")
tokenizer.fit_on_texts(train)
words = tokenizer.texts_to_matrix(train, mode = "count")
vocab_size = words.shape[1]
embedding_dim = 30
maxlen = 30
prob = 0.8

train_seq = tokenizer.texts_to_sequences(train)
test_seq = tokenizer.texts_to_sequences(test)

#spilt texts into chunks
def chunks(lst, n, minlen):
    return [lst[i:i+n] for i in range(0,len(lst),n) if len(lst[i:i+n]) > minlen]

minlen = 3
pre_train = []
ytrain = []
for i in range(len(train_seq)):
    seq_len = len(train_seq[i])
    if seq_len > maxlen:
        for chunk in chunks(train_seq[i], int(seq_len/(seq_len/maxlen + 1)), minlen):
            pre_train.append(chunk)
            ytrain.append(y_train[i])
    else:
        pre_train.append(train_seq[i])
        ytrain.append(y_train[i])
        
pre_test = []
packed_indices = {}
count = 0
for i in range(len(test_seq)):
    seq_len = len(test_seq[i])
    indices = []
    if seq_len > maxlen:
        for chunk in chunks(test_seq[i], int(seq_len/(seq_len/maxlen + 1)), minlen):
            pre_test.append(chunk)
            indices.append(count)
            count += 1
        packed_indices[i] = indices
    else:
        pre_test.append(test_seq[i])
        packed_indices[i] = [count]
        count += 1

x_train = pad_sequences(pre_train, padding = 'post', maxlen = maxlen)
x_test = pad_sequences(pre_test, padding = 'post', maxlen = maxlen)
Xtrain = np.array(x_train)
Ytrain = np.array(ytrain)
Xtrain, Ytrain = shuffle(Xtrain, Ytrain)

model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, 
                           output_dim = embedding_dim, 
                           input_length = maxlen))
model.add(LSTM(10))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam',
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

model.fit(Xtrain[:5000], Ytrain[:5000], epochs=25, verbose=True, batch_size=10)
y_test = model.predict_classes(x_test)

def allocate_class(packed_indices, predict, prob):
    predictions = []
    for _, indices in packed_indices.items():
        if(sum([predict[i] for i in indices])/len(indices) > prob):
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

test_result = allocate_class(packed_indices, [y[0] for y in y_test], prob)

output_path = '/Users/enn/Desktop/nlp/project1/project-files/test-output.json'
output = {}
for i in range(len(test_title)):
    output[test_title[i]] = {'label': int(test_result[i])}
with open(output_path, 'w') as outfile:
    json.dump(output, outfile)

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
5000/5000 [==============================] - 12s 2ms/step - loss: 0.4572 - accuracy: 0.7784
Epoch 2/25
5000/5000 [==============================] - 11s 2ms/step - loss: 0.2297 - accuracy: 0.9224
Epoch 3/25
5000/5000 [==============================] - 11s 2ms/step - loss: 0.1392 - accuracy: 0.9556
Epoch 4/25
5000/5000 [==============================] - 11s 2ms/step - loss: 0.0788 - accuracy: 0.9756
Epoch 5/25
5000/5000 [==============================] - 11s 2ms/step - loss: 0.0517 - accuracy: 0.9838
Epoch 6/25
5000/5000 [==============================] - 11s 2ms/step - loss: 0.0302 - accuracy: 0.9914
Epoch 7/25
5000/5000 [==============================] - 12s 2ms/step - loss: 0.0237 - accuracy: 0.9958
Epoch 8/25
5000/5000 [==============================] - 11s 2ms/step - loss: 0.0104 - accuracy: 0.9978
Epoch 9/25
5000/5000 [==============================] - 11s 2ms/step - loss: 0.0096 - accuracy: 0.9978
Epoch 10/25
5000/5000 [==============================] - 11s 2ms/step - l